In [1]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objects as go
import psycopg2

%matplotlib inline

In [3]:
def df_from_db(query):
    
    conn = psycopg2.connect(dbname='***', user='***', password='*********')
    cur = conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    col_names = [i[0] for i in cur.description]
    df = pd.DataFrame(rows, columns=col_names)
    conn.commit()
    cur.close()
    conn.close()

    return df


## animation with date data

In [71]:
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Twitter Geolocation Dashboard"),
    dcc.Graph(id = 'graph'),
    html.Label([
        "Date",
        dcc.Dropdown(
            id='time-dropdown', 
            options=[
            {'label': 'Dec 30', 'value': '0'},
            {'label': 'Dec 31', 'value': '1'},
            {'label': 'Jan 01', 'value': '2'}],
            value = '0'
        ),
    ]),
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("time-dropdown", "value")]
)
def update_figure(value):
    #only processing one hour here
    df_day = pd.DataFrame()
    for five_min in range(12):
        union = " union ".join([f"select * from timenpl_01_{idx+5*five_min}" for idx in range(5)])
        df = df_from_db(f"""with timenpl as ({union}) select coord_lat::numeric(10,2) * 0 + 5* {five_min} as "time", count(coord_lat) as weight, coord_lat::numeric(10,2), coord_lon::numeric(10,2), STRING_AGG(loc, '<br>') as loc , STRING_AGG("text", '<br>') as text, STRING_AGG("url", '<br>') as url from timenpl group by coord_lat::numeric(10,2), coord_lon::numeric(10,2);""")
        df_day = df_day.append(df)

    df_day['coord_lat'] = df_day['coord_lat'].astype('float64')
    df_day['coord_lon'] = df_day['coord_lon'].astype('float64')
    df_day['time'] = df_day['time'].astype('float64')

    token = "*********"
    fig = px.scatter_mapbox(df_day, lat="coord_lat", lon="coord_lon", hover_name="loc", hover_data=["text", "url"],animation_frame = 'time', zoom=1, height=600,
                        color_discrete_sequence=["green"], size= 'weight', color_continuous_scale=px.colors.cyclical.IceFire, size_max=25)


    fig.update_layout(mapbox_accesstoken=token)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    return fig

# Run app and display result inline in the notebook
app.run_server(host='0.0.0.0',debug=True)

Dash app running on http://0.0.0.0:8050/
